In [1]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:

from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from PIL import Image
#from pywaffle import Waffle
#from catboost import CatBoostClassifier, pool
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from tqdm import tqdm
import nltk
from nltk.stem import WordNetLemmatizer


In [15]:
pip install --upgrade nltk

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.5/1.5 MB 32.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [20]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 70.9/70.9 MB 36.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install squarify

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [40]:
import nltk
nltk.download('omw-1.4')
# Check if WordNet data is available and download if not
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')

# Update the NLTK data path to include the WordNet data
nltk.data.path.append('/kaggle/working/nltk_data')

# Check if stopwords data is available and download if not
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', download_dir='/kaggle/working/nltk_data')

# Update the NLTK data path to include the stopwords data
nltk.data.path.append('/kaggle/working/nltk_data')


    
from nltk.corpus import stopwords

import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import HistGradientBoostingClassifier

import plotly.express as px
import warnings


from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import SMOTE

import squarify
from sklearn.metrics import f1_score

#from nltk.corpus import wordnet

# Get synsets for the word "hello" in English WordNet


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Adarsh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:
sy =wordnet.synsets('hello')
print(sy)

[Synset('hello.n.01')]


In [44]:
synsets = wordnet.synsets('extrovert')

# Print synset information
for synset in synsets:
    print(f"Synset name: {synset.name()}")
    print(f"Definition: {synset.definition()}")
    print(f"Examples: {synset.examples()}")
    print()

Synset name: extrovert.n.01
Definition: (psychology) a person concerned more with practical realities than with inner thoughts and feelings
Examples: []

Synset name: extrovert.s.01
Definition: being concerned with the social and physical environment
Examples: []



In [45]:
def sampling(mode, df, col_name, class_names, count):
    """
    Samples the dataframe based on the mode, given class names and count.

    Parameters:
    -----------
    mode: str
        Either 'under' or 'over'
    df : pandas.DataFrame
        Input DataFrame to be undersampled
    col_name : str
        Name of the column containing the target class
    class_names : list
        List of target class names to be undersampled
    count : int
        Number of samples to be kept for each class

    Returns:
    --------
    pandas.DataFrame
        Undersampled DataFrame or Oversampled
    """
    dfs = []
    for class_name in class_names:
        if mode == 'under':
            df_class = df[df[col_name] == class_name]
            if len(df_class) > count:
                df_class = df_class.sample(count, replace=False)
            dfs.append(df_class)
        elif mode == 'over':
            df_class = df[df[col_name] == class_name]
            if len(df_class) < count:
                df_class = df_class.sample(count, replace=True)
            dfs.append(df_class)
    dfs = pd.concat(dfs)
    df_x = df[~df[col_name].isin(class_names)]
    return pd.concat([dfs,df_x])

In [46]:
def get_train(train_data):
    
    X_train = np.vstack(np.array(train_data.posts))
    y_train = np.array(train_data.type)
    
    return X_train, y_train

In [ ]:
def split(df, size):
    
    # Cleaning The Data
    df.posts, length = clear_text(df)
    
    #Splitting into train & test
    print("Splitting into train & test")
    train_data,test_data=train_test_split(df,test_size=size,random_state=0,stratify=df.type)
    
    #Applying Tfidf Vectorization
    print("Applying Tfidf Vectorization")
    vectorizer=TfidfVectorizer(max_features=5000,stop_words='english',tokenizer=Lemmatizer())
    vectorizer.fit(train_data.posts)
    
    #Applying the vectorizer transform
    train_post=vectorizer.transform(train_data.posts).toarray()
    test_post=vectorizer.transform(test_data.posts).toarray()
    
    #Label Encoding the classes as 0,1,2,3......
    print("Label Encoding the classes")
    target_encoder= LabelEncoder()
    
    #Getting the final train and test
    print("Getting the final train and test")
    train_target=target_encoder.fit_transform(train_data.type)
    test_target=target_encoder.fit_transform(test_data.type)
    print(target_encoder.classes_)
    return train_post, test_post, train_target, test_target  